In [1]:
# Import libraby
import random
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from time import sleep
from datetime import datetime
import os

# Get review counts:
def get_reviews_counts(string):
    start_index = string.find('(')
    end_index = string.find(')')
    review_count = string[start_index + 1:end_index]
    return review_count

# Create funtion to crawl
def crawl_all_ids(driver, start_url, num_pages):

    # GET INFOMATION OF ALL ITEMS
    product_ids, data_product_ids, links, title = [], [], [], []
    price, discount, review_count, quantity_sold, product_variant = [], [], [], [], []

    for page in range(1, num_pages + 1):

        # Get url
        url = f"{start_url}?p={page}"
        driver.get(url)
        sleep(random.randint(5,7))
        
        # Get product-id, data-product-id, link
        elems_link_id = driver.find_elements(By.CSS_SELECTOR, '.ProductGrid__grid .block_info_item_sp')
        product_ids = [elem.get_attribute('data-id') for elem in elems_link_id] + product_ids
        data_product_ids = [elem.get_attribute('data-product') for elem in elems_link_id] + data_product_ids
        links = [elem.get_attribute('href') for elem in elems_link_id] + links
        sleep(random.randint(5,7))

        # Get title
        elems_title = driver.find_elements(By.CSS_SELECTOR, '.ProductGrid__grid .vn_names')
        title = [elem.text for elem in elems_title] + title
        sleep(random.randint(5,7))

        # Get price
        elems_price = driver.find_elements(By.CSS_SELECTOR, '.ProductGrid__grid .item_giacu')
        price = [elem.text for elem in elems_price] + price  
        sleep(random.randint(5,7))

        # Get discount
        elems_discount = driver.find_elements(By.CSS_SELECTOR, '.ProductGrid__grid .item_giamoi')
        discount = [elem.text for elem in elems_discount] + discount
        sleep(random.randint(5,7))

        # Get review counts:
        elems_review_count = driver.find_elements(By.CSS_SELECTOR, '.ProductGrid__grid .block_count_by')
        review_count = [get_reviews_counts(elem.text) for elem  in elems_review_count] + review_count
        sleep(random.randint(5,7))

        # Get quantity_sold
        elems_quantity_sold = driver.find_elements(By.CSS_SELECTOR, '.ProductGrid__grid .item_count_by')
        quantity_sold = [elem.text.strip('\n ') for elem  in elems_quantity_sold] + quantity_sold
        sleep(random.randint(5,7))

        # Get product_variant
        elems_product_variant = driver.find_elements(By.CSS_SELECTOR, ".ProductGrid__grid .block_info_item_sp")
        product_variant = [elem.get_attribute('data-variant') for elem  in elems_product_variant] + product_variant
        sleep(random.randint(5,7))

        print(f"Scraped IDs from page {page}")
        sleep(random.randint(8,15))
    
    return product_ids, data_product_ids, links, title, price, discount, review_count, quantity_sold, product_variant

# Initialize browsser
driver = webdriver.Chrome()

# URL to crawl
start_url = "https://hasaki.vn/danh-muc/cham-soc-da-mat-c4.html?"

# Number pages to crawl
num_pages = 2

# Get data crawled
product_ids, data_product_ids, links, title, price, discount, review_count, quantity_sold, product_variant = crawl_all_ids(driver, start_url, num_pages)

# Create dataframe for data crawled
data = pd.DataFrame(
    list(zip(product_ids, data_product_ids, links, title,
        price, discount, quantity_sold, product_variant, review_count
    )),
    columns=[
        'product_id', 'data_product_id', 'link_item', 'title', 
        'original_price', 'current_price', 'quantity_sold', 'product_variant','review_count'
    ]
)

# Save into csv
current_datetime = datetime.now().strftime("%Y%m%d_%H%M")
productdata_filename = f"productdata_{current_datetime}.csv"
folder_path = "D:/HOC KI 8/3. Graduate project/hasaki_crawling"

data.to_csv(os.path.join(folder_path, "data", productdata_filename), encoding= "utf-8-sig")
print("All product IDs saved to CSV file.")

Scraped IDs from page 1
Scraped IDs from page 2
All product IDs saved to CSV file.
